# 引入參數

In [1]:
import numpy as np
M = 250
St = 60
S_avet = 53
K = 50
numsim = 10000
numrep = 20
T = 0.5
t0 = 0.25
#T-t0 = 0.25
passing_time = 0.25
r = 0.1
q = 0.04
sigma = 0.5
n = 25
dt = (T-t0)/n
u = np.exp(sigma*np.sqrt(dt))
d = np.exp(-sigma*np.sqrt(dt))
p =  (np.exp((r-q)*dt)-d) / (u-d)
print(u,d,p)

1.0512710963760241 0.951229424500714 0.4935019038548862


# 自訂函數

In [2]:
def binarysearch(A,i,j,value): #數字由大到小
    k = int(np.floor(M/2))
    up = M  #位置的邊界
    down = 0
    while (np.abs(A[i,j,k]-value)>10**(-5) ):
        #print(k)
        if (A[i,j,k]-value)> 0:
            down = max(down,k)
            if (A[i,j,k+1]-value) < 0:
                return k
            k = int(np.floor((k+up)/2) )+1
        else:
            up = min(up,k)
            k = int(np.floor((k+down)/2) )       
    return k   #這邊算出來的k是老師講義上的k-1    

#測試的數據 
i = 5
j = 1
B = np.zeros((n+1,n+1,M+1))
B[i,j,:] = M-np.array(range(M+1))
binarysearch(B,i,j,10)


240

In [3]:
def interpolsearch(A,i,j,value):
    if np.abs(A[i,j,M]-A[i,j,0]) < 10**(-6): #排除一開始差距太小
        return M
    #print(A[i,j,0],value,A[i,j,M])
    k = np.floor( M*(A[i,j,0]-value)/(A[i,j,0]-A[i,j,M]) )
    return int(k)

In [4]:
def seqsearch(A,i,j,value):
    k = 0
    while (A[i,j,k]-value) > -10**(-5)  :
        k = k+1
        if k == M:
            return M
        #print(k)
    return k-1

In [5]:
def price_interpol(A,call,i,j,Avalue):
    #k = binarysearch(A,i,j,Avalue)  #選用的方法
    #k = interpolsearch(A,i,j,Avalue)
    k = seqsearch(A,i,j,Avalue)
    # print(k)
    if k < M:
        #print(k,'<',M)
        if np.abs(A[i,j,k+1]-A[i,j,k])<10**(-3):
            return call[i,j,k]
        else:
            w = (A[i,j,k]-Avalue)/(A[i,j,k]-A[i,j,k+1])
            return w*call[i,j,k+1]+(1-w)*call[i,j,k]
    
    else: #k=m
        return call[i,j,k]
    
i = 5
j = 1   
call = np.zeros((n+1,n+1,M+1))
call[i,j,:] = M-np.array(range(M+1))
price_interpol(B,call,i,j,0)



0.0

In [6]:
def optioncal(optiontype): # 0:美式
    call = np.zeros((n+1,n+1,M+1))

    for i in range(n+1): 
        i = n-i #0~n
        for j in range(i+1):
            for k in range(M+1):
                if i == n:
                    call[n,j,k] = max(A[n,j,k]-K,0)
                    #print(A[n,j,k])
                else:
                    a = A[i,j,k]
                    Au = ( (i+1+passing_time/dt)*a+St*u**(i+1-j)*d**j )/(i+2+passing_time/dt)
                    Ad = ( (i+1+passing_time/dt)*a+St*u**(i-j)*d**(j+1) )/(i+2+passing_time/dt)

                    Cu = price_interpol(A,call,i+1,j,Au)
                    Cd = price_interpol(A,call,i+1,j+1,Ad)
                    if optiontype == 0: #美式
                        call[i,j,k] = max(A[i,j,k]-K , np.exp(-r*dt)*(p*Cu+(1-p)*Cd) )
                    else: #歐式
                        call[i,j,k]  = np.exp(-r*dt)*(p*Cu+(1-p)*Cd)
    return call[0,0,0]

# 計算部分:

## Amax、Amin、A

In [7]:
# i,j是照老師的記號並非row,column值
A_max = np.zeros((n+1,n+1))
A_min = np.zeros((n+1,n+1))
A = np.zeros((n+1,n+1,M+1))


#Amax,Amin設定好
for i in range(n+1):
    for j in range(i+1):
        A_max[i,j] =(  St *(-1+ (1-u**(i-j+1))/(1-u) + u**(i-j) *d*(1-d**j) /(1-d) ) \
                      + S_avet*(passing_time/dt+1)   ) \
                    /(i+passing_time/dt +1)
        A_min[i,j] = ( St *(-1+ (1-d**(j+1))/(1-d) + d**(j) *u*(1-u**(i-j)) /(1-u) ) \
                       + S_avet*(passing_time/dt+1)    ) \
                    /(i+passing_time/dt +1)

              
#A的資料，總共有M+1個資料
for i in range(n+1):
    for j in range(i+1):
        A[i,j,0] = A_max[i,j]
        A[i,j,M] = A_min[i,j]
        #print(i,j)
        for k in range(1,M):
            A[i,j,k] = (M-k)/M *A_max[i,j] + k/M *A_min[i,j]


## option price

In [8]:
print("歐式:",optioncal(1))

歐式: 6.553765634276815


In [9]:
print("美式:",optioncal(0))

美式: 7.05929863958554


# Monte Carlo

In [10]:
#n = 5
#numsim = 10
def MCsimpath(St,r,q,sigma,dt,numsim,n,K):
    stockpath = np.zeros((numsim,n))
    for i in range(n):  #共n期
        if i == 0:
            stockpath[:,i] = np.random.normal(loc=np.log(St)+(r-q - sigma**2 /2)*dt, scale=sigma*np.sqrt(dt),size =numsim)
        else:
            stockpath[:,i] = np.random.normal((r-q - sigma**2 /2)*dt, sigma*np.sqrt(dt),numsim) + stockpath[:,i-1]
    stockpath = np.exp(stockpath)
    
    futureprice = (np.sum(stockpath,axis=1)+S_avet*(passing_time/dt+1))/(n+passing_time/dt+1) -K #axis=1這個軸的座標變動，而其他軸固定
    futureprice[futureprice<0] = 0
    #print(futureputprice)
    return np.mean(np.exp(-r*(T-t0))*futureprice) 

mcresult = []
for i in range(numrep):
    mcresult.append(MCsimpath(St,r,q,sigma,dt,numsim,n,K))

pmn = np.mean(mcresult)
psd = np.std(mcresult)
print('Monte Carlo的值:%f \n95%%信賴區間:(%f,%f)' %(pmn,pmn-psd*2,pmn+psd*2))


Monte Carlo的值:6.573092 
95%信賴區間:(6.498312,6.647873)


# BONUS 1

In [11]:
A = np.zeros((n+1,n+1,M+1))
#A的資料，總共有M+1個資料
for i in range(n+1):
    for j in range(i+1):
        A[i,j,0] = A_max[i,j]
        A[i,j,M] = A_min[i,j]
        #print(i,j)
        for k in range(1,M):
            A[i,j,k] =np.exp( (M-k)/M *np.log(A_max[i,j]) + k/M *np.log(A_min[i,j]) )
print("歐式:",optioncal(1))
print("美式:",optioncal(0))

歐式: 6.553770286255998
美式: 7.05929887621928


# BONUS 2
* 1.sequential search
* 2.Binary search
* 3.linear search

In [12]:
def seqsearch(A,i,j,value):
    k = 0
    while (A[i,j,k]-value) > -10**(-5)  :
        k = k+1
        if k == M:
            return M
        #print(k)
    return k-1
    
#測試的數據 
i = 5
j = 1
B = np.zeros((n+1,n+1,M+1))
B[i,j,:] = M-np.array(range(M+1))
seqsearch(B,i,j,249)

1

In [13]:
def interpolsearch(A,i,j,value):
    if np.abs(A[i,j,M]-A[i,j,0]) < 10**(-6): #排除一開始差距太小
        return M
    #print(A[i,j,0],value,A[i,j,M])
    k = np.floor( M*(A[i,j,0]-value)/(A[i,j,0]-A[i,j,M]) )
    return int(k)
#測試的數據 
i = 5
j = 1
B = np.zeros((n+1,n+1,M+1))
B[i,j,:] = M-np.array(range(M+1))
interpolsearch(B,i,j,240)

10